<a href="https://colab.research.google.com/github/M-adithya-0925/ml-project/blob/main/cnn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from google.colab import drive
import os

# Mount your Google Drive to access the dataset
drive.mount('/content/drive')

# Define the paths to your dataset directory in Google Drive
dataset_dir = '/content/drive/MyDrive/plant ml/Indian Medicinal Leaves Image Datasets'
img_width, img_height = 150, 150  # You can adjust these dimensions

# Define the number of classes (types of plants)
num_classes = len(os.listdir(dataset_dir))  # Automatically count the subfolders

# Create an ImageDataGenerator to preprocess and augment your data
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split 20% of the data for validation
)

# Load the dataset using flow_from_directory
generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Use the training subset
)

validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use the validation subset
)

# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

# Train the model
model.fit(
    generator,
    steps_per_epoch=generator.samples // 32,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32
)

# Save the model for future use
model.save('/content/drive/My Drive/plant_model.h5')

# Now, you can use this trained model to identify plants

# Load the model
model = tf.keras.models.load_model('/content/drive/My Drive/plant_model.h5')

# Use the model to identify a plant from an image
from tensorflow.keras.preprocessing import image
import numpy as np

def identify_plant(image_path):
    img = image.load_img(image_path, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize the image

    result = model.predict(img)
    class_indices = generator.class_indices
    inv_map = {v: k for k, v in class_indices.items()}
    predicted_class = inv_map[np.argmax(result)]

    return predicted_class

from flask import Flask, render_template, request, redirect, url_for
import os

app = Flask(__name)

@app.route('/')
def index():
    return render_template('relcam.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        return redirect(request.url)
    if file:
        img = image.load_img(file, target_size=(img_width, img_height))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = img / 255.0  # Normalize the image

        result = model.predict(img)
        class_indices = generator.class_indices
        inv_map = {v: k for k, v in class_indices.items()}
        predicted_class = inv_map[np.argmax(result)]

        return f'The identified plant is: {predicted_class}'

if __name__ == '__main__':
    app.run()

